In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import HashingVectorizer
import pickle
import mysql.connector

In [3]:
#update run_config set value='2020-01-11 04:41:59';
#update new_anomalies set status = 'N', train_flag = 'N';

"select stack, count from noise_data_summary where from_unixtime(unix_timestamp(insert_time,'yyyy/MM/dd hh:mm:ss')) > from_unixtime(unix_timestamp('2020/03/16 04:57:00','yyyy/MM/dd hh:mm:ss'))"

In [18]:
#mydb.close()
mydb = mysql.connector.connect(
    host="localhost",
    user="jenkins",
    passwd="ariba",
    database="ariba_logs",
    buffered=True
)

data = []




In [37]:
def getCurrentTime(mydb):
    timeSql = "select date_format(now(),'%Y-%m-%d %H:%i:%S')"
    timeCursor = mydb.cursor()
    timeCursor.execute(timeSql)
    timeData = timeCursor.fetchone()
    timeCursor.close()
    currentTime = timeData[0]
    return currentTime




In [38]:
def getPrevRunTime(mydb):
    timeSql = "select value from run_config where name = 'MODEL_TRAIN_LAST_RUN'"
    timeCursor = mydb.cursor()
    timeCursor.execute(timeSql)
    timeData = timeCursor.fetchone()
    prevRunTime = timeData[0]
    timeCursor.close()
    return prevRunTime


In [39]:
def getUnTrainedRecs(mydb, prevRunTime):
    sqlData = []
    trainCursor = mydb.cursor()
    selectSql = "SELECT  log_str FROM new_anomalies where status = 'N'and time <= STR_TO_DATE(%s,'%Y-%m-%d %H:%i:%S') order by id"
    whereValues = (prevRunTime,)
    trainCursor.execute(selectSql,whereValues)
    if trainCursor.rowcount > 0 :
        sqlData = trainCursor.fetchall()
    trainCursor.close()
    data = list(map(lambda el: el[0], sqlData))
    return data

In [40]:
def updateRunConfigValues(mydb,currentTime):
    updateConfigCursor = mydb.cursor()
    updateSql = "UPDATE run_config set value = %s where name = 'MODEL_TRAIN_LAST_RUN'"
    updateValues = (currentTime,)
    updateConfigCursor.execute(updateSql,updateValues )
    updateConfigCursor.close()
    mydb.commit()

In [41]:
def updateProcessedRecords(mydb, prevRunTime, currentTime):
    updateAnomalyFlagsCursor = mydb.cursor()
    updateFlagSql = "update new_anomalies set train_flag = 'Y' where time >= STR_TO_DATE(%s,'%Y-%m-%d %H:%i:%S') and time <= STR_TO_DATE(%s,'%Y-%m-%d %H:%i:%S')"
    updateFlagValues = (prevRunTime, currentTime,)
    updateAnomalyFlagsCursor.execute(updateFlagSql, updateFlagValues)
    updateAnomalyFlagsCursor.close()
    mydb.commit()


In [42]:
Valid952HostnameRegex = "[a-zA-Z0-9\.]+"
vectorizer  = CountVectorizer(token_pattern=Valid952HostnameRegex)


In [43]:
from sklearn.metrics.pairwise import cosine_similarity
currentTime = getCurrentTime(mydb)
prevRunTime = getPrevRunTime(mydb)


In [44]:
data = getUnTrainedRecs(mydb, currentTime)
tfidf_matrix = vectorizer.fit_transform(data)

In [45]:
pickle.dump(vectorizer.vocabulary_,open("ExceptionDB.pkl","wb"))


In [46]:
import scipy.sparse
scipy.sparse.save_npz('./ExceptionSparse.npz',tfidf_matrix)

In [47]:
updateRunConfigValues(mydb,currentTime)
#updateProcessedRecords(mydb, prevRunTime, currentTime)

# Section of Code for Comparing inbox data with Model

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import HashingVectorizer
import pickle
import mysql.connector
from pyhive import hive

In [23]:
import scipy.sparse
Valid952HostnameRegex = "[a-zA-Z0-9\.]+"
vectorizer = CountVectorizer(token_pattern=Valid952HostnameRegex, decode_error="replace",vocabulary=pickle.load(open("ExceptionDB.pkl", "rb")))
vectorizer._validate_vocabulary()
tfidf_matrix = scipy.sparse.load_npz('./ExceptionSparse.npz')

In [24]:
mydb = mysql.connector.connect(
    host="localhost",
    user="jenkins",
    passwd="ariba",
    database="ariba_logs",
    buffered=True
)

In [25]:
def getPrevRunTime(mydb):
    timeSql = "select value from run_config where name = 'MODEL_TRAIN_LAST_RUN'"
    timeCursor = mydb.cursor()
    timeCursor.execute(timeSql)
    timeData = timeCursor.fetchone()
    prevRunTime = timeData[0]
    timeCursor.close()
    return prevRunTime


In [26]:
getPrevRunTime(mydb)

'2020-03-13 01:25:35'

In [27]:
def getTrainedRecs(mydb, prevRunTime):
    sqlData = []
    trainCursor = mydb.cursor()
    selectSql = "SELECT  log_str FROM new_anomalies where status = 'N'and time <= STR_TO_DATE(%s,'%Y-%m-%d %H:%i:%S') order by id"
    whereValues = (prevRunTime,)
    trainCursor.execute(selectSql,whereValues)
    if trainCursor.rowcount > 0 :
        sqlData = trainCursor.fetchall()
    trainCursor.close()
    data = list(map(lambda el: el[0], sqlData))
    return data

In [28]:
# Take all the trained records until the last training time. 
prevRunTime = getPrevRunTime(mydb)
data = getTrainedRecs(mydb,prevRunTime)

In [29]:
vectorizer.vocabulary

{'data.ariba.com': 0,
 'data2.ariba.com': 1,
 's4.ariba.com': 6,
 'java.ariba.com': 5,
 'honeybee.ariba.com': 4,
 'honey.ariba.com': 3,
 'data34565.ariba.com': 2}

In [30]:
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter 
#cosine_similarities = linear_kernel(unseen_tfidf[0:1], tfidf_matrix).flatten()
#cosine_similarities = pairwise_kernels(unseen_tfidf[0:1],
#                 tfidf_matrix,
#                 metric='cosine').flatten()
#related_docs_indices = cosine_similarities.argsort()[:-5:-1]

In [31]:
def getCurrentTime(mydb):
    timeSql = "select date_format(now(),'%Y-%m-%d %H:%i:%S')"
    timeCursor = mydb.cursor()
    timeCursor.execute(timeSql)
    timeData = timeCursor.fetchone()
    timeCursor.close()
    currentTime = timeData[0]
    return currentTime


In [32]:
def getInboxProcessPrevRunTime(mydb):
    timeSql = "select value from run_config where name = 'INBOX_PROCESS_LAST_RUN'"
    timeCursor = mydb.cursor()
    timeCursor.execute(timeSql)
    timeData = timeCursor.fetchone()
    if timeCursor.rowcount > 0:
        prevRunTime = timeData[0]
        timeCursor.close()
        return prevRunTime
    else:
        timeCursor.close()
        return ''

In [33]:
def getInboxRecs(preRunTime) :
    conn = hive.Connection(host="localhost", port=10000, username="YOU")
    query = "select stack, count from noise_data_summary where from_unixtime(unix_timestamp(insert_time,'yyyy/MM/dd hh:mm:ss')) > from_unixtime(unix_timestamp('{}','yyyy/MM/dd hh:mm:ss'))".format('2020/03/16 04:57:00')
    cursor = conn.cursor()
    cursor.execute(query)
    data = cursor.fetchall()
    conn.close()
    return data
    

In [34]:
def getInboxRecsMySql(mydb, prevRunTime):
    sqlData = []
    trainCursor = mydb.cursor()
    selectSql = "SELECT  log_str, count FROM new_anomalies_inbox where status = 'N'and time >= STR_TO_DATE(%s,'%Y-%m-%d %H:%i:%S') order by id"
    whereValues = (prevRunTime,)
    trainCursor.execute(selectSql,whereValues)
    if trainCursor.rowcount > 0 :
        sqlData = trainCursor.fetchall()
    trainCursor.close()
    #data = list(map(lambda el: el[0], sqlData))
    return sqlData

In [35]:
def updateInboxProcessRunConfigValues(mydb,currentTime):
    updateConfigCursor = mydb.cursor()
    updateSql = "UPDATE run_config set value = %s where name = 'INBOX_PROCESS_LAST_RUN'"
    updateValues = (currentTime,)
    updateConfigCursor.execute(updateSql,updateValues )
    updateConfigCursor.close()
    mydb.commit()

In [36]:
def updateInboxRecords(mydb, prevRunTime, currentTime):
    updateAnomalyFlagsCursor = mydb.cursor()
    updateFlagSql = "update new_anomalies_inbox set status = 'Y' where time >= STR_TO_DATE(%s,'%Y-%m-%d %H:%i:%S') and time <= STR_TO_DATE(%s,'%Y-%m-%d %H:%i:%S')"
    updateFlagValues = (prevRunTime, currentTime,)
    updateAnomalyFlagsCursor.execute(updateFlagSql, updateFlagValues)
    updateAnomalyFlagsCursor.close()
    mydb.commit()

In [37]:
def joinInboxRecords(data):
    res = list(Counter(key for key, num in data  
                  for idx in range(num)).items()) 
    return res

In [38]:
currentTime = getCurrentTime(mydb)
prevInboxProcessTime = getInboxProcessPrevRunTime(mydb)

In [40]:
sqlData = getInboxRecs(prevInboxProcessTime)

In [42]:
inboxRecs = dict(joinInboxRecords(sqlData))

In [43]:
unseen_data = list(inboxRecs.keys())


In [44]:
print(vectorizer.vocabulary.keys())

dict_keys(['data.ariba.com', 'data2.ariba.com', 's4.ariba.com', 'java.ariba.com', 'honeybee.ariba.com', 'honey.ariba.com', 'data34565.ariba.com'])


In [45]:
tfidf_matrix

<8x7 sparse matrix of type '<class 'numpy.int64'>'
	with 15 stored elements in Compressed Sparse Row format>

In [46]:
# This is the data that we pick up from the hive table which is not fetched for processing
# Store the time until which the processing is last done. 
# From the last time stamp, pick the records unti now


#unseen_data = ['data.ariba.com data2.ariba.com','data.ariba.com', 's4.ariba.com java.ariba.com','honeybee.ariba.com']


unseen_tfidf = vectorizer.transform(unseen_data)
unseen_tfidf.shape
cosine_similarity(unseen_tfidf, tfidf_matrix)

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [47]:
cosine_similarities = pairwise_kernels(unseen_tfidf,
                 tfidf_matrix,
                 metric='cosine')

In [48]:
print("Before the start : {}".format(unseen_tfidf.shape))
most_matching_index = -1
sim_matrix = cosine_similarities.argsort(axis=-1)
print(sim_matrix)
print(cosine_similarities)
counter = 0;
best_sim_value = 0;
for sim_array in sim_matrix:
    most_matching_index = sim_array[:-2:-1][0]
    best_sim_value = cosine_similarities[counter][most_matching_index]
    print("The best similarity value is ---> {} for {}".format(best_sim_value ,unseen_data[counter]) )
    best_sim_value = round(best_sim_value,4)
    
    if best_sim_value < 0.9 :
        
        # Before this step, verify if there is any record in the new_anomalies having the train_flag = 'N'
        # If yes, add the count.
        selectCursor = mydb.cursor( )
        selectValues = (str(unseen_data[counter]),)
        selectSql = "SELECT ID, count FROM new_anomalies WHERE log_str = %s"
        selectCursor.execute(selectSql, selectValues)
        
        if selectCursor.rowcount > 0 :
            print('Count Value --------------- {}'.format(selectCursor.rowcount))
            selectResult = selectCursor.fetchall()
            updateCursor = mydb.cursor()
            for (i,cnt) in selectResult:
                updateStmt = 'update new_anomalies set count = count+ %s where id = %s'
                new_rec_count = inboxRecs[unseen_data[counter]];
                updateValues = (int(new_rec_count),int(i))
                updateCursor.execute(updateStmt,updateValues)
            updateCursor.close()
            mydb.commit()    
            selectCursor.close()        
        else:
            mycursor = mydb.cursor()
            inSql = "INSERT INTO new_anomalies(log_str, count, status) VALUES(%s, %s, 'N')"
            new_rec_count = inboxRecs[unseen_data[counter]];
            val = (unseen_data[counter], int(new_rec_count))
            mycursor.execute(inSql, val)
            mydb.commit()
            mycursor.close()
            print(mycursor.rowcount, "record inserted.")
            
            
    else:
        #print('-----------------------------------------')
        sim_value_array = cosine_similarities[counter]
        sim_index_array = sim_array[::-1]
        for index in sim_index_array:
            sim_value = round(sim_value_array[index],2)
            
                
            if sim_value >= 0.9:
                selectCursor = mydb.cursor( )
                selectValues = (str(data[index]),)
                selectSql = "SELECT ID FROM new_anomalies WHERE log_str = %s"
                selectCursor.execute(selectSql, selectValues)
                if selectCursor.rowcount > 0 :
                    selectResult = selectCursor.fetchall()
                    if sim_value == 1.0:
                        updateCursor = mydb.cursor()
                        
                        for i in selectResult:
                            updateStmt = 'update new_anomalies set count = count+ %s where id = %s'
                            new_rec_count = inboxRecs[unseen_data[counter]];
                            updateValues = (int(new_rec_count),int(i[0]))
                            updateCursor.execute(updateStmt,updateValues)
                    else:
                        selectCursor.execute(selectSql, selectValues)
                        selectResult = selectCursor.fetchone()
                        insertSql = "insert into new_anomaly_relations(rel_log_str, id) values(%s, %s)"
                        insertCursor = mydb.cursor()
                        values = (data[index], int(selectResult[0]))
                        insertCursor.execute(insertSql, values)
                        insertCursor.close()
                        mydb.commit()
                        
                    updateCursor.close()
                    mydb.commit()    
                                
                selectCursor.close()
            else:
                break
    counter = counter+1    
        
print("At the completion : {}".format(unseen_tfidf.shape))    


Before the start : (18, 7)
[[0 1 2 3 4 5 6 7]
 [0 1 2 3 4 5 6 7]
 [0 1 2 3 4 5 6 7]
 [0 1 2 3 4 5 6 7]
 [0 1 2 3 4 5 6 7]
 [0 1 2 3 4 5 6 7]
 [0 1 2 3 4 5 6 7]
 [0 1 2 3 4 5 6 7]
 [0 1 2 3 4 5 6 7]
 [0 1 2 3 4 5 6 7]
 [0 1 2 3 4 5 6 7]
 [0 1 2 3 4 5 6 7]
 [0 1 2 3 4 5 6 7]
 [0 1 2 3 4 5 6 7]
 [0 1 2 3 4 5 6 7]
 [0 1 2 3 4 5 6 7]
 [0 1 2 3 4 5 6 7]
 [0 1 2 3 4 5 6 7]]
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
The best similarity value is ---> 0.0 for  ariba.common.core.condition.archesobjectmatchingquery
-1 record inserted.
The best similarity value 

In [49]:
updateInboxRecords(mydb,prevInboxProcessTime, currentTime)

In [111]:
updateInboxProcessRunConfigValues(mydb, currentTime)

In [51]:
mydb.commit()

In [52]:
mydb.close()

In [61]:
print("Before the start : {}".format(unseen_tfidf.shape))
most_matching_index = -1
sim_matrix = cosine_similarities.argsort(axis=-1)
print(sim_matrix)
print(cosine_similarities)
counter = 0;
best_sim_value = 0;
for sim_array in sim_matrix:
    most_matching_index = sim_array[:-2:-1][0]
    best_sim_value = cosine_similarities[counter][most_matching_index]
    best_sim_value = round(best_sim_value,2)
    print("The best similarity value is {} for {}".format(best_sim_value ,unseen_data[counter]) )
    if best_sim_value < 0.9 :
        
        # Before this step, verify if there is any record in the new_anomalies having the train_flag = 'N'
        # If yes, add the count.
        
        mycursor = mydb.cursor()
        inSql = "INSERT INTO new_anomalies(log_str, count, status) VALUES(%s, %s, 'E')"
        val = (unseen_data[counter], 1)
        mycursor.execute(inSql, val)
        mydb.commit()
        mycursor.close()
        print(mycursor.rowcount, "record inserted.")
    else:
        print('-----------------------------------------')
        sim_value_array = cosine_similarities[counter]
        sim_index_array = sim_array[::-1]
        for index in sim_index_array:
            sim_value = round(sim_value_array[index],2)
            
                
            if sim_value >= 0.9:
                selectCursor = mydb.cursor( )
                print('{} : {} : {} : {}'.format(unseen_data[counter], index, sim_value, data[index]))
                selectValues = (str(data[index]),)
                #selectSql = "SELECT ID FROM new_anomalies WHERE log_str = '{}'".format(data[index])
                selectSql = "SELECT ID FROM new_anomalies WHERE log_str = %s"
                print('----------------------->{}<-----'.format(data[index]))
                selectCursor.execute(selectSql, selectValues)
                print('*****************************{}'.format(selectCursor.rowcount))
                if selectCursor.rowcount > 0 :
                    selectResult = selectCursor.fetchall()
                    if sim_value > 1.0:
                        updateCursor = mydb.cursor()
                        
                        for id in selectResult:
                            updateStmt = 'update new_anomalies set count = count+ %d where id = %d'
                            updateValues = (int(10),int(id[0]))
                            updateCursor.execute(updateStmt,updateValues)
                    else:
                        print('Value is less than 1.0')
                        insertSql = "insert into new_anomaly_relations(rel_log_str, id) values(%s, %s)"
                        insertCursor = mydb.cursor()
                        values = (data[index], int(id[0]))
                        print('--------- ------------- {}'.format(id[0]))
                        insertCursor.execute(insertSql, values)
                        mycursor.close()
                        mydb.commit()
                    
                    mydb.commit()    
                                
                selectCursor.close()
            else:
                break
    counter = counter+1    
        
print("At the completion : {}".format(unseen_tfidf.shape))    



Before the start : (4, 5)
[[2 3 4 5 1 0]
 [2 3 4 5 0 1]
 [0 1 3 4 5 2]
 [0 1 2 3 4 5]]
[[1.         0.70710678 0.         0.         0.         0.        ]
 [0.70710678 1.         0.         0.         0.         0.        ]
 [0.         0.         1.         0.         0.         0.        ]
 [0.         0.         0.         1.         1.         1.        ]]
The best similarity value is 1.0 for data.ariba.com data2.ariba.com
-----------------------------------------
data.ariba.com data2.ariba.com : 0 : 1.0 : data.ariba.com data2.ariba.com
----------------------->data.ariba.com data2.ariba.com<-----
*****************************1
Value is less than 1.0


TypeError: 'builtin_function_or_method' object is not subscriptable

In [28]:
print(loaded_vectorizer.vocabulary.keys())

dict_keys(['data.ariba.com', 'data2.ariba.com', 'honeybee.ariba.com', 's4.ariba.com', 'java.ariba.com'])


In [29]:


mycursor.execute("SELECT * FROM new_anomalies where status= 'N'")
myresult = mycursor.fetchall()
for x in myresult:
  print(x)


NameError: name 'mycursor' is not defined

In [30]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

In [31]:
loaded_sparse.shape

(9, 5)

In [32]:
y_train = ['Noise','Noise','Alert','Alert','Alert','Noise','Noise','Alert']
clf = MultinomialNB().fit(loaded_sparse, y_train)

ValueError: Found input variables with inconsistent numbers of samples: [9, 8]

In [33]:
print(clf.predict(loaded_vectorizer.transform(['data.ariba.com data2.ariba.com'])))

NameError: name 'clf' is not defined

In [34]:
clf

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [39]:
mydb.commit()

In [57]:
trainCursor.rowcount

-1